In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
file_path = "XAUUSD_H1_202201022300_202410252300.csv"
df = pd.read_csv(file_path, delimiter="\t")
df.columns = [col.strip('<>') for col in df.columns]
df.columns
column_name = 'CLOSE'

In [4]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[[column_name]])

In [5]:
lookback_window = 10 
X, y = [], []
for i in range(len(df_scaled) - lookback_window - 10): 
    X.append(df_scaled[i:i+lookback_window, 0])  
    y.append(df_scaled[i+lookback_window+10, 0])  

X = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\Albaloo Rayaneh\AppData\Local\Temp\ipykernel_22300\1692864067.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(device)


In [ ]:
class CNN1DModel(nn.Module):
    def __init__(self):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.fc1 = nn.Linear(128 * (lookback_window - 4), 64)  
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = CNN1DModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_train_pred = model(X_batch)
        train_loss = criterion(y_train_pred.squeeze(), y_batch)
        train_loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        y_test_preds, y_tests = [], []
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_test_pred = model(X_batch)
            y_test_preds.append(y_test_pred.squeeze())
            y_tests.append(y_batch)

        y_test_pred = torch.cat(y_test_preds)
        y_test = torch.cat(y_tests)
        test_loss = criterion(y_test_pred, y_test)

      
    if (epoch + 1) % 10 == 0:
        r2_train = r2_score(y_train.cpu().numpy(), model(X_train).detach().cpu().squeeze().numpy())
        r2_test = r2_score(y_test.cpu().numpy(), y_test_pred.cpu().numpy())
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss.item():.4f}, Test Loss: {test_loss.item():.4f}, R2 Train: {r2_train:.4f}, R2 Test: {r2_test:.4f}')


Epoch [10/100], Training Loss: 0.0002, Test Loss: 0.0002, R2 Train: 0.9904, R2 Test: 0.9828
Epoch [20/100], Training Loss: 0.0002, Test Loss: 0.0010, R2 Train: 0.9854, R2 Test: 0.9190
Epoch [30/100], Training Loss: 0.0002, Test Loss: 0.0004, R2 Train: 0.9897, R2 Test: 0.9675
Epoch [40/100], Training Loss: 0.0002, Test Loss: 0.0003, R2 Train: 0.9904, R2 Test: 0.9719
Epoch [50/100], Training Loss: 0.0001, Test Loss: 0.0002, R2 Train: 0.9907, R2 Test: 0.9827
Epoch [60/100], Training Loss: 0.0001, Test Loss: 0.0003, R2 Train: 0.9887, R2 Test: 0.9792
Epoch [70/100], Training Loss: 0.0001, Test Loss: 0.0004, R2 Train: 0.9895, R2 Test: 0.9704
Epoch [80/100], Training Loss: 0.0000, Test Loss: 0.0003, R2 Train: 0.9902, R2 Test: 0.9757
Epoch [90/100], Training Loss: 0.0001, Test Loss: 0.0002, R2 Train: 0.9914, R2 Test: 0.9833
Epoch [100/100], Training Loss: 0.0001, Test Loss: 0.0003, R2 Train: 0.9906, R2 Test: 0.9739


In [8]:
model.eval()
with torch.no_grad():
    y_train_pred = model(X_train).cpu()
    y_test_pred = model(X_test).cpu()

y_train_pred = scaler.inverse_transform(y_train_pred.squeeze().numpy().reshape(-1, 1))
y_train = scaler.inverse_transform(y_train.cpu().numpy().reshape(-1, 1))
y_test_pred = scaler.inverse_transform(y_test_pred.squeeze().numpy().reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.cpu().numpy().reshape(-1, 1))

In [9]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(len(y_train))),
    y=y_train.flatten(),
    mode='lines',
    name='Train Data',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=list(range(len(y_train))),
    y=y_train_pred.flatten(),
    mode='lines',
    name='Train Prediction',
    line=dict(color='cyan', dash='dot')
))

fig.add_trace(go.Scatter(
    x=list(range(len(y_train), len(y_train) + len(y_test))),
    y=y_test.flatten(),
    mode='lines',
    name='Test Data',
    line=dict(color='green')
))

fig.add_trace(go.Scatter(
    x=list(range(len(y_train), len(y_train) + len(y_test))),
    y=y_test_pred.flatten(),
    mode='lines',
    name='Test Prediction',
    line=dict(color='red', dash='dot')
))

fig.update_layout(
    title="Train and Test Data vs Predictions",
    xaxis_title="Index",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

fig.show()
